In [3]:
import numpy as np
import pickle

import ngboost
from ngboost.distns import MultivariateNormal
from ngboost.scores import LogScore
from sklearn.tree import DecisionTreeRegressor

In [2]:
class predictor:
    """
    Attributes inherited from all classes
    __init__ fills the class based on a dictionary
    save pickles itself, this is for convience and one does not need to convert
    self.fname to a filename etc.
    """

    def __init__(self, params={}):
        for key in params.keys():
            setattr(self, key, params[key])

    def save(self, fname="model_1"):
        ## Use fname if the fname is defined.
        pickle.dump(self, open(getattr(self, "fname", fname) + ".p", "wb"))
        

class mvn_predictor(predictor):
    """
    Template, nothing defined as mvn predictors have custom treatments
    """

    def summary(self):
        pass

In [4]:
DEFAULT_BASE = {
    "criterion": "friedman_mse",
    "min_samples_leaf": 1,
    "min_samples_split": 2,
    "min_weight_fraction_leaf": 0.0,
    "max_depth": 3,
    "splitter": "best",
}

DEFAULT_NGBOOST = {"minibatch_frac": 1.0, "learning_rate": 0.01}

MAX_ITER = 1000

class mvn_ngboost(mvn_predictor):
    def __init__(self, params={}):
        super().__init__(params)
        self.model = None
        self.base = None
        if not hasattr(self, "params_base"):
            self.params_base = DEFAULT_BASE
        if not hasattr(self, "params_ngboost"):
            self.params_ngboost = DEFAULT_NGBOOST

    def fit(self, X_train, Y_train, X_valid, Y_valid, base=None, retrain=False):
        p = Y_train.shape[1]
        dist = MultivariateNormal(p)

        if base is None:
            b2 = DecisionTreeRegressor(**self.params_base)
            self.base = b2
            print(self.params_base)
        else:
            b2 = base
            self.base = base
        n_estimators = MAX_ITER
        self.model = ngboost.NGBoost(
            Dist=dist,
            Score=LogScore,
            Base=b2,
            **self.params_ngboost,
            verbose_eval=10,
            n_estimators=n_estimators
        )

        self.model.fit(
            X=X_train,
            Y=Y_train,
            X_val=X_valid,
            Y_val=Y_valid,
            early_stopping_rounds=100,
        )
        self.model.best_val_loss_itr
        if retrain:
            self.model = ngboost.NGBoost(
                Dist=dist,
                Score=LogScore,
                Base=b2,
                **self.params_ngboost,
                verbose_eval=10,
                n_estimators=self.model.best_val_loss_itr + 1
            )

            X = np.vstack([X_train, X_valid])
            Y = np.vstack([Y_train, Y_valid])
            self.model.fit(X=X, Y=Y)

    def scipy_distribution(self, X_test, cmat_output=False):
        preds = self.model.pred_dist(X_test, max_iter=self.model.best_val_loss_itr)
        if cmat_output:
            out = [preds.mean(), preds.cov]
        else:
            out = preds.scipy_distribution()
        return out

In [ ]:
# To-do: write a function to check residual norm

residual_norm = (Y_test[:,1] - Y_dists.mean())/Y_dists.std()
print(np.mean(residual_norm), np.std(residual_norm))
standard_normal = np.random.randn(residual_norm.size)
plt.hist(standard_normal, bins=50, alpha=0.5, density=True)
plt.hist(residual_norm, bins=50, alpha=0.5, density=True)
plt.show()
# plt.yscale('log')